# **Bert + catboost**

In [6]:
!pip install catboost

In [2]:
from catboost import CatBoostClassifier
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from catboost import CatBoostClassifier, Pool, metrics, cv
from tqdm import tqdm
import numpy as np
import re
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch.optim as optim
import matplotlib.pyplot as plt
import wandb
from transformers import get_cosine_schedule_with_warmup


import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:
# Скачиваем и обрабатываем данные
train_data = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')  
train_data = train_data[['text', 'target']] 


train_data.loc[train_data['text'] == 'like for the music video I want some real action shit like burning buildings and police chases not some weak ben winston shit', 'target'] = 1
train_data.loc[train_data['text'] == 'Hellfire is surrounded by desires so be careful and donÛªt let your desires control you! #Afterlife', 'target'] = 1
train_data.loc[train_data['text'] == 'To fight bioterrorism sir.', 'target'] = 1
train_data.loc[train_data['text'] == '.POTUS #StrategicPatience is a strategy for #Genocide; refugees; IDP Internally displaced people; horror; etc. https://t.co/rqWuoy1fm4', 'target'] = 1
train_data.loc[train_data['text'] == 'CLEARED:incident with injury:I-495  inner loop Exit 31 - MD 97/Georgia Ave Silver Spring', 'target'] = 1
train_data.loc[train_data['text'] == '#foodscare #offers2go #NestleIndia slips into loss after #Magginoodle #ban unsafe and hazardous for #humanconsumption', 'target'] = 1
train_data.loc[train_data['text'] == 'In #islam saving a person is equal in reward to saving all humans! Islam is the opposite of terrorism!', 'target'] = 1
train_data.loc[train_data['text'] == 'Who is bringing the tornadoes and floods. Who is bringing the climate change. God is after America He is plaguing her\n \n#FARRAKHAN #QUOTE', 'target'] = 1
train_data.loc[train_data['text'] == 'RT NotExplained: The only known image of infamous hijacker D.B. Cooper. http://t.co/JlzK2HdeTG', 'target'] = 0
train_data.loc[train_data['text'] == "Mmmmmm I'm burning.... I'm burning buildings I'm building.... Oooooohhhh oooh ooh...", 'target'] = 1
train_data.loc[train_data['text'] == "wowo--=== 12000 Nigerian refugees repatriated from Cameroon", 'target'] = 0
train_data.loc[train_data['text'] == "He came to a land which was engulfed in tribal war and turned it into a land of peace i.e. Madinah. #ProphetMuhammad #islam", 'target'] = 1
train_data.loc[train_data['text'] == "Hellfire! We donÛªt even want to think about it or mention it so letÛªs not do anything that leads to it #islam!", 'target'] = 1
train_data.loc[train_data['text'] == "The Prophet (peace be upon him) said 'Save yourself from Hellfire even if it is by giving half a date in charity.'", 'target'] = 1
train_data.loc[train_data['text'] == "Caution: breathing may be hazardous to your health.", 'target'] = 0
train_data.loc[train_data['text'] == "I Pledge Allegiance To The P.O.P.E. And The Burning Buildings of Epic City. ??????", 'target'] = 1
train_data.loc[train_data['text'] == "#Allah describes piling up #wealth thinking it would last #forever as the description of the people of #Hellfire in Surah Humaza. #Reflect", 'target'] = 1
train_data.loc[train_data['text'] == "that horrible sinking feeling when youÛªve been at home on your phone for a while and you realise its been on 3G this whole time", 'target'] = 0


test_data = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [27]:
class Bert(nn.Module):
    def __init__(self, trainable = False):
        super().__init__()
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.bert = AutoModel.from_pretrained('distilbert-base-uncased').to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
        if trainable == False:
            for m in self.bert.modules():
                for name, params in m.named_parameters():
                    params.requires_grad = False
        self.target_indx = 0
        

    def forward(self, input):

        encoding = self.tokenizer.batch_encode_plus(
            input,  # List of input texts
            padding="max_length",
            max_length=512,  # Pad to the maximum sequence length
            truncation=True,  # Truncate to the maximum sequence length if necessary
            return_tensors='pt',  # Return PyTorch tensors
            add_special_tokens=True  # Add special tokens CLS and SEP
        )

        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)

        out = self.bert(input_ids, attention_mask, output_hidden_states=True)
        out = out[0][:,self.target_indx,:]

        return out
    
class dataset_bilder(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
    
    
class bert_catboost(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.bert = Bert()
        self.catboost = CatBoostClassifier(**params)
        
    def train_catboost(self, train, val, batch_train_size, batch_val_size):
        bert_output = []
        bert_output_val = []
        target = []
        target_val = []
        
        for X_train, y_train in tqdm(train, desc="Train", colour="CYAN"):
            if y_train.shape[0] == batch_train_size:
                bert_output.append(self.bert(X_train).tolist())
                target.append(y_train.tolist())
            
        for X_val, y_val in tqdm(val, desc="Val", colour="CYAN"):
            if y_val.shape[0] == batch_val_size:
                bert_output_val.append(self.bert(X_val).tolist())
                target_val.append(y_val.tolist())
        
        bert_output = np.array(bert_output)
        bert_output = bert_output.reshape(-1, bert_output.shape[-1])
        bert_output_val = np.array(bert_output_val)
        bert_output_val = bert_output_val.reshape(-1, bert_output_val.shape[-1])
        
        target = np.array(target)
        target = target.reshape(-1)
        target_val = np.array(target_val)
        target_val = target_val.reshape(-1)
        
        self.catboost.fit(
            bert_output, target,
            eval_set=(bert_output_val, target_val),
            #logging_level='Verbose',  # you can uncomment this for text output
            plot=True
        );
        
        
    def forward(self, input):
        output = self.bert(input).tolist()
        output = self.catboost.predict(output)
        
        return output

    
class model_testment(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        
    def take_accuracy(self,test_data):
        first_iter = True
        for X, y in tqdm(test_data, desc="Testment", colour="CYAN"):
            if first_iter == True:
                model_prediction = self.model(X)
                target = y.numpy() 
                first_iter = False
            else:
                model_prediction = np.concatenate((model_prediction, self.model(X)), axis = 0)
                target = np.concatenate((target, y.numpy()), axis = 0)
        
        return len(model_prediction[model_prediction == target])/len(model_prediction)
    
    def create_result(self, pd_test):
        texts = pd_test['text'].values.tolist()   
        model_output = []
        for text in texts:
            model_output.append(self.model([text])[0])
        model_output = np.array(model_output)
        
        submission = pd.DataFrame({'id': pd_test['id'], 'target': model_output})
        
        return submission
    
    def forward(self, input):
        return self.model(input)

In [28]:
from sklearn.model_selection import train_test_split  # To split data into training and testing sets
#Train-test split

X_train, X_val, y_train, y_val = train_test_split(train_data.text.values.tolist(), train_data.target.values.tolist(), test_size=0.2)

train_dataset = dataset_bilder(X_train, y_train)
val_dataset = dataset_bilder(X_val, y_val)

batch_train_size = 32
batch_val_size = 64

train = DataLoader(
        train_dataset,
        batch_size=batch_train_size,
        num_workers=4,
        shuffle=True,
        collate_fn=None,
    )

val = DataLoader(
        val_dataset,
        batch_size=batch_val_size,
        num_workers=4,
        shuffle=False,
        collate_fn=None,
    )

In [29]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

params = {
    'iterations': 500,
    'learning_rate': 0.1,
    'eval_metric': metrics.Accuracy(),
    'random_seed': 42,
    'logging_level': 'Silent',
    'use_best_model': True,
    'task_type' : 'GPU'
}

model = bert_catboost(params)
model.train_catboost(train, val, batch_train_size, batch_val_size)

acc = model_testment(model).take_accuracy(val)
print('accuracy: ' + str(acc))

submission = model_testment(model).create_result(test_data)
submission.to_csv('submission.csv', index=False)

Val: 100%|██████████| 24/24 [00:26<00:00,  1.09s/it]


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [30]:
acc = model_testment(model).take_accuracy(val)
print('accuracy: ' + str(acc))

Testment: 100%|██████████| 24/24 [00:26<00:00,  1.12s/it]

accuracy: 0.8073089700996677


In [12]:
submission = model_testment(model).create_result(test_data)
submission.to_csv('submission.csv', index=False)

#  **DistilBertForSequenceClassification**

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Bert_classification(nn.Module): # создаем класс с бертом и токенайзером
    def __init__(self):
        super().__init__()
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.bert = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2).to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

    def forward(self, input):
        encoding = self.tokenizer.batch_encode_plus(
            input,  # List of input texts
            padding="max_length",
            max_length=512,  # Pad to the maximum sequence length
            truncation=True,  # Truncate to the maximum sequence length if necessary
            return_tensors='pt',  # Return PyTorch tensors
            add_special_tokens=False  # Add special tokens CLS and SEP
        )

        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)

        out = self.bert(input_ids, attention_mask, output_hidden_states=True)

        return out.logits
    
class dataset_bilder(Dataset): # для создания датасета
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    
class model_usage(): # создем класс с кодом для обучения, тестирования и сохранению результата
    def __init__(self, model):
        self.model = model 
    
    def train(self, dataloader, optimizer, loss_func, scheduler, epochs):
        self.model.bert.train()
        for _ in range(epochs):
            for texts, labels in tqdm(dataloader, desc="Epoch", colour="GREEN"):
                labels = labels.to(device)
                optimizer.zero_grad()
                output = self.model(texts)
                labels = labels.long()
                loss = loss_func(output, labels)
                wandb.log({"loss_val": loss})
                loss.backward()
                optimizer.step()
                scheduler.step()
        

    def test(self, dataloader, loss_func):
        self.model.eval()
        testloss, correct = 0, 0
        num_batches = len(dataloader)
        size = len(dataloader.dataset)
        
        with torch.no_grad():
            for texts, labels in tqdm(dataloader, desc="Eval", colour="CYAN"):
                labels = labels.to(device)
                output = self.model(texts)
                labels = labels.long()
                loss = loss_func(output, labels)
                testloss += loss.item()
                preds = torch.argmax(output, dim=1)
                correct += (preds == labels).type(torch.float).sum().item()

        correct /= size
        testloss /= num_batches

        print(f"Testment: \nAccuracy: {(100*correct):>0.1f}%, Avg loss: {testloss:>8f} \n")
        
    def create_result(self, pd_test):
        texts = pd_test['text'].values.tolist()   
        model_output = []
        for text in tqdm(texts, desc="Creating", colour="CYAN"):
            model_output.append(torch.argmax(self.model([text])).item())
        model_output = np.array(model_output)
        
        submission = pd.DataFrame({'id': pd_test['id'], 'target': model_output})
        
        return submission

In [ ]:
#Сплитаем дату
from sklearn.model_selection import train_test_split  # To split data into training and testing sets

X_train, X_val, y_train, y_val = train_test_split(train_data.text.values.tolist(), train_data.target.values.tolist(), test_size=0.2)

#Создаем даталоадер для обучения
train_dataset = dataset_bilder(X_train, y_train)
val_dataset = dataset_bilder(X_val, y_val)

batch_train_size = 32
batch_val_size = 64

train = DataLoader(
        train_dataset,
        batch_size=batch_train_size,
        num_workers=4,
        shuffle=True,
        collate_fn=None,
    )

val = DataLoader(
        val_dataset,
        batch_size=batch_val_size,
        num_workers=4,
        shuffle=False,
        collate_fn=None,
    )
# Инициализируем модель
model = Bert_classification()

# Создаем вещи для обучения
epochs = 5
optimizer = optim.Adam(model.bert.parameters(), lr=5e-5)
loss_func = nn.CrossEntropyLoss()
scheduler = get_cosine_schedule_with_warmup(
    optimizer, num_warmup_steps=200, 
    num_training_steps= 191 * epochs  # 191 - число батчей в датасете
)

wandb.login()
wandb.init(
            project="DistilBertForSequenceClassification",
            config={
                "config": 'Tuning',
            })

# Обучаем
model_f = model_usage(model)
model_f.train(train, optimizer, loss_func, scheduler, epochs)                            
Bert_model = model_f.model

# Тестируем
model_f.test(val, loss_func)

# Сохраняем результат на тест дате
submission = model_f.create_result(test_data)
submission.to_csv('submission.csv', index=False)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


loss_val,▆▆▄▆▃█▅▄▄▄▃▂▂▂▃▃▁▄▂▃▂▃▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▂
loss_val,0.18071


Epoch:  59%|█████▊    | 112/191 [02:34<01:48,  1.38s/it]